In [5]:
import transformers
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML
from datasets import load_dataset

In [6]:
# log into huggingface account
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
# load your own dataset
datasets = load_dataset("text", data_files={"train": path_to_train.txt, "validation": path_to_validation.txt}

In [ ]:
# model checkpoint; performing casual language modeling (predicting the next word)
model_checkpoint = "distilgpt2"

# tokenizer
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

In [ ]:
# tokenize on all the texts 
def tokenize_function(examples):
    return tokenizer(examples["text"])

In [ ]:
# apply function to all the splits
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])